In [21]:
import numpy as np
import matplotlib.pyplot as plt
import pysptools.util as util
import pysptools.eea as eea
import pysptools.classification as clssf
import pysptools.abundance_maps as amap
import pysptools.noise.dnoise as denoise
import os

In [22]:
dataCubes = np.load('meteoriteDataCubes.npy').item()
backgroundCubes = np.load('backgroundDataCubes.npy').item()

In [23]:
def addBackgroundColumnsToData(): 
    for sample in dataCubes: 
        BlackBackground = np.repeat([[backgroundCubes['CallibrationReadingFar']]], len(dataCubes[sample]), axis=0); 
        Velvet = np.repeat([[backgroundCubes['VelvetFar']]], len(dataCubes[sample]), axis=0); 
        dataWithBackground = np.concatenate((dataCubes[sample],BlackBackground,Velvet), axis=1)
        dataCubesWithBackground[sample] = dataWithBackground

In [24]:
dataCubesWithBackground = {}
addBackgroundColumnsToData()

In [25]:
def getNormalizationMaxVals(cube):
    samples = []
    for sample in cube:
        for row in cube[sample]:
            samples.append(row)
    samples = np.reshape(samples,(867,6))
    return np.max(np.transpose(samples), axis=1)
     
    #np.reshape(samples, (51,17,6))
        

        

In [26]:
NormalizationMaxVals = getNormalizationMaxVals(dataCubesWithBackground)
print NormalizationMaxVals

[  393.26     15.995   830.29    947.27     23.955  1293.715]


In [27]:
def NormalizeCubePerColumn(cube):
    normalized_cube = cube 
    print NormalizationMaxVals
    for sample in normalized_cube: 
        for i, row in enumerate(normalized_cube[sample]): 
            normalized_cube[sample][i] = normalized_cube[sample][i] /  NormalizationMaxVals
    #print cube
    return normalized_cube
print dataCubesWithBackground['Allende']
normalizedDataCubesWithBackground = NormalizeCubePerColumn(dataCubesWithBackground)      
print normalizedDataCubesWithBackground['Allende']
#print normalizedDataCubesWithBackground['Toluca']

[[[  78.97          6.86        148.87        106.85          3.12         58.2       ]
  [  89.34666667    7.24        166.22333333  126.01666667    3.12         92.15      ]
  [ 176.88666667    9.14        290.08        295.69666667    9.02333333
    468.42      ]
  [ 193.13333333    9.9         353.82333333  354.27333333    9.71666667
    495.09666667]
  [ 201.25333333    9.9         359.47        356.40333333   10.06333333
    519.75      ]
  [ 226.97666667   11.04666667  411.51666667  411.78         12.15333333
    668.07333333]
  [ 252.69333333   11.04666667  416.35666667  436.62666667   13.88666667
    730.31333333]
  [ 248.63666667   11.04666667  551.91666667  513.66         11.80333333
    526.21333333]
  [ 161.99666667    9.52        388.52        330.13          6.94333333
    257.85      ]
  [ 143.49333333    8.          277.17        219.02          5.55666667
    229.15666667]
  [ 264.42666667   11.04666667  463.15666667  480.64333333   14.58
    766.68666667]
  [ 208.473

In [ ]:
#display spectral lines 
#for sample in dataCubes: 
#    util.display_linear_stretch(dataCubes[sample],1,3,5)
    #spectral_lines = util.convert2d(sample); 

In [28]:
#Taken from: https://www.quora.com/How-do-I-generate-n-visually-distinct-RGB-colours-in-Python
def get_spaced_colors(n):
    max_value = 16581375 #255**3
    interval = int(max_value / n)
    colors = [hex(I)[2:].zfill(6) for I in range(0, max_value, interval)]
    
    return [(int(i[:2], 16), int(i[2:4], 16), int(i[4:], 16)) for i in colors]

In [29]:
def ATGP(sample):  
    ATGP = eea.ATGP()
    ATGP.extract(sample,q=2)
    return ATGP.get_idx()
    #ATGP.display()


In [30]:
def NFINDR(sample): 
    NFINDR = eea.NFINDR()
    NFINDR.extract(sample, q=2)
    return NFINDR.get_idx()
    #NFINDR.display()
    

In [31]:
def FIPPI(sample):
    FIPPI = eea.FIPPI()
    FIPPI.extract(sample, q=2)
    return FIPPI.get_idx()
    #NFINDR.display()


In [32]:
def PPI(sample):
    PPI = eea.PPI()
    PPI.extract(sample, q=2)
    return PPI.get_idx()
    #NFINDR.display()


In [33]:
#Endmembers per row
def endmembersPerRow(cube):
    for sample in cube: 
        print sample + ':'
        for i, row in enumerate(cube[sample]): 
            print "row " + str(i) + ":"  
            print "row shape: " + str(np.shape(row))
            print ATGP(np.array([row]))
            print NFINDR(np.array([row]))
            print PPI(np.array([row]))
 


In [34]:
#Endmembers per sample
ATGP_dict = {}
NFINDR_dict = {}
PPI_dict = {}


for sample in dataCubesWithBackground: 
        ATGP_dict[sample] = ATGP(dataCubesWithBackground[sample])
        NFINDR_dict[sample] = NFINDR(dataCubesWithBackground[sample])
        PPI_dict[sample] = PPI(dataCubesWithBackground[sample])
        
 


In [35]:
#endmembers per sample where endmember algorithms are run with normalized data 

ATGP_normalized_dict = {}
NFINDR_normalized_dict = {}
PPI_normalized_dict = {}
for sample in normalizedDataCubesWithBackground: 
        ATGP_normalized_dict[sample] = ATGP(normalizedDataCubesWithBackground[sample])
        NFINDR_normalized_dict[sample] = NFINDR(normalizedDataCubesWithBackground[sample])
        PPI_normalized_dict[sample] = PPI(normalizedDataCubesWithBackground[sample])

In [19]:
endmember_dicts = {'ATGP':ATGP_dict, 'NFINDR':NFINDR_dict, 'PPI':PPI_dict}
endmember_normalized_dicts = {'ATGP':ATGP_normalized_dict, 'NFINDR':NFINDR_normalized_dict, 'PPI':PPI_normalized_dict}
#np.save('endmember_dicts.npy', endmember_dicts)
#np.save('endmember_normalized_dicts.npy', endmember_normalized_dicts)

In [45]:
 
colors = get_spaced_colors(100) 
band = np.array([610,680,730,760,810,860])
endmember_colors = ['yellow','orange','lime']

def SpectralPlotsWithEndmembers(cube,algorithm,my_endmember_dict):
    endmember_dict = my_endmember_dict[algorithm]; 
    background = np.array(backgroundCubes['CallibrationReadingFar']) / NormalizationMaxVals
    for sample in cube: 
        #print sample
        linearized = util.convert2d(cube[sample])       
        for i, (a,b) in enumerate(endmember_dict[sample]): 
            plt.plot(band,cube[sample][b][a],color=endmember_colors[i], linewidth='8')
            #print str((a,b)) + ': ' + endmember_colors[i]
        plt.plot(band,np.transpose(linearized))
        plt.title(sample)
        plt.xlabel('Wavelength (nm)')
        plt.ylabel('Intensity (counts)')
        print backgroundCubes['CallibrationReadingFar']
        print background
        plt.plot(band,background,color='black',linewidth='3')
        #plt.plot(band,backgroundCubes['VelvetFar']/getNormalizationMaxVals(cube),color='black',linewidth='3')
        #plt.show()
        plt.figure(num=1, figsize=(8, 6))
        plt.savefig('../Results/EndmembersNormalized/' + algorithm + '/' + sample)
        plt.clf()

print endmember_normalized_dicts['ATGP']['Soku']
#SpectralPlotsWithEndmembers(normalizedDataCubesWithBackground, 'ATGP',endmember_normalized_dicts)
#SpectralPlotsWithEndmembers(normalizedDataCubesWithBackground, 'NFINDR',endmember_normalized_dicts)
#SpectralPlotsWithEndmembers(normalizedDataCubesWithBackground, 'PPI',endmember_normalized_dicts)
#SpectralPlotsWithEndmembers(dataCubesWithBackground, 'ATGP',endmember_dicts)
SpectralPlotsWithEndmembers(dataCubesWithBackground, 'NFINDR',endmember_dicts)
#SpectralPlotsWithEndmembers(dataCubesWithBackground, 'PPI',endmember_dicts)

[(3, 0), (10, 0)]
[  75.41955556    6.86        140.18488889  100.78888889    3.05066667
   62.10644444]
[ 0.19178039  0.42888403  0.16883846  0.10639933  0.12734989  0.04800628]
[  75.41955556    6.86        140.18488889  100.78888889    3.05066667
   62.10644444]
[ 0.19178039  0.42888403  0.16883846  0.10639933  0.12734989  0.04800628]
[  75.41955556    6.86        140.18488889  100.78888889    3.05066667
   62.10644444]
[ 0.19178039  0.42888403  0.16883846  0.10639933  0.12734989  0.04800628]
[  75.41955556    6.86        140.18488889  100.78888889    3.05066667
   62.10644444]
[ 0.19178039  0.42888403  0.16883846  0.10639933  0.12734989  0.04800628]
[  75.41955556    6.86        140.18488889  100.78888889    3.05066667
   62.10644444]
[ 0.19178039  0.42888403  0.16883846  0.10639933  0.12734989  0.04800628]
[  75.41955556    6.86        140.18488889  100.78888889    3.05066667
   62.10644444]
[ 0.19178039  0.42888403  0.16883846  0.10639933  0.12734989  0.04800628]
[  75.41955556  

In [ ]:
#DECISION: USE ENDMEMBERS SUPPLIED BY NFINDR ALGORITHM 
fully_constrained_abundance_map_dict = {}

def makeAbundanceMaps(cube,endmember_dicts):
    for sample in cube:
        data = cube[sample]
        endmember_indices = NFINDR_dict[sample]
        endmembers = np.array([data[b][a] for (a,b) in endmember_indices])

        unmixed_unconstrained = amap.UCLS()
        unconstrained_abundance_map = unmixed_unconstrained.map(data, endmembers) 

        unmixed_nonnegative_constrained = amap.NNLS()
        nonnegative_abundance_map = unmixed_nonnegative_constrained.map(data, endmembers) 

        unmixed_fully_constrained = amap.FCLS()
        fully_constrained_abundance_map = unmixed_fully_constrained.map(data, endmembers) 
        fully_constrained_abundance_map_dict[sample] = fully_constrained_abundance_map
        #os.mkdir('./' + sample)
        #unmixed_fully_constrained.plot('./' + sample)
makeAbundanceMaps(normalizedDataCubesWithBackground,endmember_normalized_dicts)

In [ ]:
#small hack to determine which endmember corresponds to the black background based on me 
#manually adding a column of black pixels to the data array

def whichisblack(abundance_map):
    black = 0
    #print "which is black? " + str(abundance_map[0][16][0]) + 'vs.' + str(abundance_map[0][16][1])
    if abundance_map[0][16][0] > 0.94:
        return 0
    elif abundance_map[0][16][1] > 0.94: 
        return 1
    else:
        return "error"

In [ ]:
#NEXT DECISION: USE FULLY CONSTRAINED MAP (IE NON NEGATIVE AND SUMS TO 1)

pure_signal_dict = {}
black_endmember = backgroundCubes['CallibrationReadingFar'] 
normalized_black_endmember = backgroundCubes['CallibrationReadingFar'] / NormalizationMaxVals
def extractSignalFromBackground(cube,endmember_dict,my_black_endmember):
    for sample in cube: 
        #note that this dict is set based on how makeabundancemap function above is run (ie/normalized vs not)
        abundance_map = fully_constrained_abundance_map_dict[sample]
        endmember_indices = endmember_dict[sample]
        data = dataCubesWithBackground[sample]
        endmembers = np.array([data[b][a] for (a,b) in endmember_indices])

        #print abundance_map
        black = whichisblack(abundance_map)
        #print black
        if black == 0:
            signal = 1
        else:
            signal = 0

        pixels = []
        for i, row in enumerate(cube[sample]): 
            for j, pixel in enumerate(row): 
                #print pixel
                #note how below I'm using my far reading endmember for all samples. Could be worth correcting
                #but based on the plots I've seen, the black readings are close enough to each other relative to signal
                #that I'm not extremely worried for my first round of analysis 
                #print i
                #print j
                #print signal
                if abundance_map[i][j][signal] > 0.75:
                    extracted_signal_in_pixel = (cube[sample][i][j] - abundance_map[i][j][black] * my_black_endmember ) / abundance_map[i][j][signal]
                    pixels.append(extracted_signal_in_pixel)
        pure_signal_dict[sample] = pixels

extractSignalFromBackground(normalizedDataCubesWithBackground,endmember_normalized_dicts['NFINDR'],normalized_black_endmember)  

    
    

In [ ]:
np.save('pure_signal_dict.npy', pure_signal_dict)  

In [ ]:
colors = get_spaced_colors(100) 
band = np.array([610,680,730,760,810,860])
#highestthreshold: 0.85
#highthreshold: 0.75
#lowthreshold: 0.5
def plotExtractedSignals(pure_signal_dict):
    print len(pure_signal_dict)
    for sample in pure_signal_dict:
        plt.clf()
        data = pure_signal_dict[sample]
        print sample
        print np.shape(data)
        plt.plot(band,np.transpose(data))
        plt.title(sample)
        #plt.ylim([0,1])
        #plt.show()
        #os.mkdir('./' + sample)
        plt.savefig('../Results/NormalizedExtractedSignalsHighThreshold/' + sample + '.pdf')
         

plotExtractedSignals(pure_signal_dict)
#plotPureSignals(pure_signal_dict)
#NOTE: HORRIBLE HORRIBLE RESULTS!!! Maybe try just non negative constrained? 
#DECISION FOR NOW: USE CLASSIFICATION INSTEAD 

In [ ]:
purest_pixels_dict = {}

def populatePurestPixelsDict(cube):
    classification = clssf.AbundanceClassification()
    for sample in cube:
        abundancemap = fully_constrained_abundance_map_dict[sample]
        classified_pixels = classification.classify(abundancemap,0.3)

        black = classified_pixels[0][16]
        if black == 2: 
            signal = 1
        else:
            signal = 2
        pixel_array = []
        for i,row in enumerate(classified_pixels): 
            for j,column in enumerate(row):
                if classified_pixels[i][j] == signal: 
                    pixel_array.append(cube[sample][i][j])
        purest_pixels_dict[sample] = pixel_array
    #classification.plot('./ExtractedSignals/' + sample + '/')
#print np.shape(dataCubesWithBackground['Allende'])
#print np.shape(purest_pixels_dict['Allende'])
populatePurestPixelsDict(normalizedDataCubesWithBackground)

In [ ]:
def plot_purest_pixels(purest_pixels_dict):
    for sample in purest_pixels_dict: 
        plt.plot(band,np.transpose(purest_pixels_dict[sample]))
        #plt.show()
        plt.savefig('../Results/NormalizedPurestPixels2/' + sample)
        plt.clf()
        
plot_purest_pixels(purest_pixels_dict)

In [ ]:
np.save('purest_pixels_dict.npy', purest_pixels_dict)

In [1]:
samplelist = []
def getNormalizationMaxVals(purest_pixels_dict):
    for sample in purest_pixels_dict:
        for row in purest_pixels_dict[sample]:
            samplelist.append(row)
    return np.array(np.amax(samplelist, axis=0))
print getNormalizationMaxVals(purest_pixels_dict)

NameError: name 'purest_pixels_dict' is not defined

In [ ]:
def plot_purest_pixels_normalized(purest_pixels_dict):
    max_vals = NormalizationMaxVals
    #print np.shape(max_vals)
    for sample in purest_pixels_dict: 
        normalized_data = np.array(purest_pixels_dict[sample]) / np.array(max_vals)
        normalized_data_transpose = np.transpose(normalized_data)
        #print np.shape(normalized_data_transpose)
        plt.plot(band,normalized_data_transpose)
        #plt.show()
        #os.mkdir('./Results/PurestPixelsNormalized/' + sample)
        #plt.savefig('../Results/LateNormalizedPurestPixels2/' + sample)
        plt.clf()
        
plot_purest_pixels_normalized(purest_pixels_dict)

In [ ]:
a = [1000,2000,3000,4000,5000,6000],[1000,2000,3000,4000,5000,6000]  
#print np.shape(a)
#print np.shape([getNormalizationMaxVals(purest_pixels_dict)])
print "max vals: " + str(getNormalizationMaxVals(purest_pixels_dict))
data = getNormalizationMaxVals(purest_pixels_dict)
print np.divide(a,data)